# Task 1: Housing Price Regression

This task is focused on predicting the price of a house listing based on a pre-determined set of features. This notebook is structured as follows:

1. Setup
2. Exploratory Data Analysis (EDA)
3. Model Evaluation
4. Prediction

# 1. Setup

The following section is required to setup this notebook. Import the appropriate modules and modify model hyperparameters here. 

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import ticker
from IPython.display import display

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

#import torch
#import torch.nn as nn
#import torch.optim as optim
#from torch.utils.data import Dataset,DataLoader

import re
from wangsheng_utils import *

In [2]:
# Hyperparameters

RANDOM_CONTROL = 42 # For reproducibility
BEST_MODEL = 'rf' # Choose best regression model based on evaluation
TRAIN_SIZE = 0.8 # Fraction of training set used for model evaluation; the remaining split is validation set
RADIUS_OF_INFLUENCE_KM = 0.5 # Radius of interest for a location centered on their geographical coordinates

# Random Forest: Fill in based on GridSearch results
RF_NUM_ESTIMATORS = 50
RF_MAX_DEPTH = 40
RF_MAX_FEATURES = 2
RF_MIN_SPLIT = 3
RF_MIN_LEAF = 1
RF_BOOTSTRAP = False
RF_CRITERION = "squared_error"

# Gradient Boosting: Fill in based on GridSearch results
GB_NUM_ESTIMATORS = 50
GB_MAX_DEPTH = 50
GB_CRITERION = "squared_error"
GB_LEARNING_RATE = 0.01

# AdaBoost: Fill in based on GridSearch results
AB_NUM_ESTIMATORS = 50
AB_MAX_DEPTH = 30
AB_SPLITTER = "best"
AB_LEARNING_RATE = 0.01

# Extra Trees Regressor: Fill in based on GridSearch results
ET_NUM_ESTIMATORS = 50
ET_MAX_DEPTH = 40
ET_MAX_FEATURES = 2
ET_MIN_SPLIT = 3
ET_MIN_LEAF = 1
ET_BOOTSTRAP = False
ET_CRITERION= "squared_error"

# Bagging Regressor: Fill in based on GridSearch results
BR_NUM_ESTIMATORS = 100
BR_MAX_DEPTH = 50
BR_SPLITTER = "best"
BR_LEARNING_RATE = 0.1

# Neural Net: Fill in based on test iterations
NN_NUM_EPOCHS = 10
NN_BATCH_SIZE = 32
NN_LEARNING_RATE = 0.1

In [3]:
# Define model training methods
def train(X_feature, y_label, model_name='rf'):
    model = None
    if model_name=='lr':
        model = LinearRegression().fit(X_feature, y_label)
    elif model_name=='rf':
        model = RandomForestRegressor(n_estimators=RF_NUM_ESTIMATORS, criterion=RF_CRITERION, max_depth=RF_MAX_DEPTH,
                      min_samples_split=RF_MIN_SPLIT, min_samples_leaf=RF_MIN_LEAF, max_features=RF_MAX_FEATURES,
                      bootstrap=RF_BOOTSTRAP, random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    elif model_name=='gb':
        model = GradientBoostingRegressor(n_estimators=GB_NUM_ESTIMATORS, learning_rate=GB_LEARNING_RATE,
                               max_depth=GB_MAX_DEPTH, criterion=GB_CRITERION, 
                               random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    elif model_name=='ab':
        dt = DecisionTreeRegressor(max_depth=AB_MAX_DEPTH, splitter=AB_SPLITTER,
                                   random_state=RANDOM_CONTROL)
        model = AdaBoostRegressor(base_estimator=dt, n_estimators=AB_NUM_ESTIMATORS, learning_rate=AB_LEARNING_RATE,
                              random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    elif model_name=='et':
        model = ExtraTreesRegressor(n_estimators=ET_NUM_ESTIMATORS, criterion=ET_CRITERION, max_depth=ET_MAX_DEPTH,
                              min_samples_split=ET_MIN_SPLIT, min_samples_leaf=ET_MIN_LEAF, max_features=ET_MAX_FEATURES,
                              bootstrap=ET_BOOTSTRAP, random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    elif model_name=='br':
        dt = DecisionTreeRegressor(max_depth=BG_MAX_DEPTH, splitter=BG_SPLITTER,
                                   random_state=RANDOM_CONTROL)
        model = BaggingRegressor(base_estimator=dt, n_estimators=BR_NUM_ESTIMATORS, learning_rate=BR_LEARNING_RATE,
                             random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    elif model_name=='nn':
        pass
    else:
        pass

    return model
            
# Define model prediction method
def predict(model, X_feature) -> pd.DataFrame: # Fix it. Numpy array is returned
    y_hat = model.predict(X_feature)
    return y_hat

# 2. Exploratory Data Analysis (EDA)

We first explore our given training data. This is done via the following sub-tasks:

- Visualize
- Pre-process
- Post-process

## 2a. Visualize

Here, we take our first look at the data to get an idea of how the attribute values are spread.

In [4]:
# Read training data
df = pd.read_csv('data/train.csv') 
print(df.shape)
df.head()

(20254, 21)


,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
0,122881,hdb flat for sale in 866 yishun street 81,sembawang / yishun (d27),866 yishun street 81,hdb 4 rooms,NaN,1988.0,3.0,2.0,1115,...,unspecified,NaN,116.0,https://www.99.co/singapore/hdb/866-yishun-str...,1.414399,103.837196,0,yishun south,yishun,514500.0
1,259374,hdb flat for sale in 506b serangoon north aven...,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,99-year leasehold,1992.0,4.0,2.0,1575,...,unspecified,"1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbserangoon-e...,1.372597,103.875625,0,serangoon north,serangoon,995400.0
2,665422,4 bed condo for sale in meyerhouse,128 meyer road,meyerhouse,condo,freehold,2022.0,4.0,6.0,3070,...,partial,"studio, 3, 4, 5, 6 br",56.0,https://www.99.co/singapore/condos-apartments/...,1.298773,103.895798,0,mountbatten,marine parade,8485000.0
3,857699,3 bed condo for sale in leedon green,26 leedon heights,leedon green,Condo,freehold,2023.0,3.0,2.0,958,...,partial,"studio, 1, 2, 3, 4 br",638.0,https://www.99.co/singapore/condos-apartments/...,1.312364,103.803271,0,farrer court,bukit timah,2626000.0
4,216061,2 bed condo for sale in one bernam,1 bernam street,one bernam,condo,99-year leasehold,2026.0,2.0,1.0,732,...,unspecified,"studio, 1, 2, 3, 4, 5 br",351.0,https://www.99.co/singapore/condos-apartments/...,1.273959,103.843635,0,anson,downtown core,1764000.0


In [5]:
def visualize():
    plt.figure(figsize=(10, 10))
    plt.tick_params(labelsize=10)
    df['price_per_square_ft'] = df['price']/df['size_sqft']
    
    filter_price_sqft_hdb = ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) & (df.title.str.contains('hdb','Hdb', regex=False)))
    df[filter_price_sqft_hdb][['property_type','num_baths','num_beds','size_sqft','price','price_per_square_ft']]
    plot1 = df[filter_price_sqft_hdb][['property_type','num_baths','num_beds','size_sqft','price','price_per_square_ft']]
    
    plt.title("HDB price per square feet($/ft2) vs square feet(ft2)")
    plt.scatter(plot1['price_per_square_ft'],plot1['size_sqft'])
    plt.xlabel('price per square ft ($/ft2)', fontsize=16)
    plt.ylabel('size_sqft (ft2)', fontsize=16)
    plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.gca().xaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().xaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.tight_layout()
    plt.show()
    
    
    filter_1 = ((df['price_per_square_ft'] < 10000) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) & (df.title.str.contains('hdb','Hdb', regex=False))))
    
    plt.figure(figsize=(10, 10))
    plt.tick_params(labelsize=10)
    plot2 = df[filter_1][['property_type','num_baths','num_beds','size_sqft','price','price_per_square_ft']]
    plt.title("HDB price per square feet($/ft2) vs square feet(ft2)")
    plt.scatter(plot2['price_per_square_ft'],plot2['size_sqft'])
    plt.xlabel('price per square ft ($/ft2)', fontsize=16)
    plt.ylabel('size_sqft (ft2)', fontsize=16)
    plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.gca().xaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().xaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.tight_layout()
    plt.show()
    
    filter_2 = ((df['price_per_square_ft'] < 2000) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) & (df.title.str.contains('hdb','Hdb', regex=False))))
    
    plt.figure(figsize=(10, 10))
    plt.tick_params(labelsize=10)
    plot3 = df[filter_2][['property_type','num_baths','num_beds','size_sqft','price','price_per_square_ft']]
    plt.title("HDB price per square feet($/ft2) vs square feet(ft2)")
    plt.scatter(plot3['price_per_square_ft'],plot3['size_sqft'])
    plt.xlabel('price per square ft ($/ft2)', fontsize=16)
    plt.ylabel('size_sqft (ft2)', fontsize=16)
    plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.gca().xaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().xaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.tight_layout()
    plt.show()
    
    
    filter_3 = ((df['price_per_square_ft'] > 1400) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) & (df.title.str.contains('hdb','Hdb', regex=False))))
    
    table1 = df[filter_3][['title','property_type','num_baths','num_beds','size_sqft','price','price_per_square_ft']].sort_values(by=['price_per_square_ft'])
    pd.options.display.float_format = '{:.0f}'.format
    display(table1)

## 2b. Pre-process

Here, we go through the training data and use the information gained in the section above to pick and choose what attributes we wish to consider in our analysis as well as their representation. 

We call this _pre-processing_ as we are working on the whole training set here. To avoid information leakage, we ensure that we do not look at the label data unless required to check for invalid inputs(more on this below). Additionally, we do not perform any aggregation or imputation here to ensure that the validation set does not influence our training set for the following sections.

The _pre-processing_ is comprised of the following tasks which are performed in order:

- Remove invalid values
- Remove outliers
- Handle missing values
- Data Transform
- Data Augmentation
- Ignore attributes

TODO: Talk about each attribute here. 

For HDB,

Assume that num_beds refers only to the bedrooms excluding the living rooms
* Hdb 2-room = 1 bedroom , 1 bathroom
* Hdb 3-room = 2 bedroom , 2 bathroom
* Hdb 4-room = 3 bedroom , 2 bathroom
* Hdb 5-room = 3 bedroom , 2 bathroom
* Hdb 3-gen = 4 bedroom , 3 bathroom
* Hdb Executive = 3/4 bedroom, 2/3 bathroom
* Hdb Masionette = 3/4 bedroom, 2/3 bathroom
* Hdb Jumbo = 4 bedroom, 4 bathroom

References:
* http://www.data.com.sg/template-m.jsp?p=my/1.html 
* https://www.hdb.gov.sg/residential/buying-a-flat/finding-a-flat/types-of-flats

OneMap:

* https://www.onemap.gov.sg/docs/


In [6]:
def remove_invalid_values(df) -> pd.DataFrame:
    df = df.copy()
    # Price is the target regression variable. If negative or 0, treat that row as invalid (logical assumption)
    if 'price' in df:
        df = df[df.price > 0]

    # Filter out HDB prices more than 2,000,000 (real-world knowledge; ref. source)
    if 'price' in df:
        filter_price_hdb = ((df.price > 2000000) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
        df_dropped = df[filter_price_hdb]
        df = df.drop(df[filter_price_hdb].index)
        print("Filterout HDB prices more than 2,000,000")
        display(df_dropped)
        
    # Filter out HDB with more bathrooms than bedrooms (real-world knowledge; ref. source)
    filter_bath_beds_hdb = ((df.num_baths > df.num_beds) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df_dropped = df[filter_bath_beds_hdb]
    df = df.drop(df[filter_bath_beds_hdb].index)
    print("Filter out HDB with more bathrooms than bedrooms")
    display(df_dropped)

    # Filter out HDB with more than 4 bathrooms or 6 bedrooms (real-world knowledge; ref. source)
    filter_bath_beds_4_hdb = (((df.num_baths > 4) | (df.num_beds > 6)) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df_dropped = df[filter_bath_beds_4_hdb]
    df = df.drop(df[filter_bath_beds_4_hdb].index)
    print("Filter out HDB with more than 4 bathrooms or 6 bedrooms")
    display(df_dropped)
    
    return df

In [7]:
def remove_outliers(df) -> pd.DataFrame:
    df = df.copy()
    # Filter out HDB with size > 2500 sqft (outlier detection)
    filter_size_hdb = ((df.size_sqft > 2500) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df[filter_size_hdb][['property_type','num_baths','num_beds','size_sqft','price']]
    df_dropped = df[filter_size_hdb]
    df = df.drop(df[filter_size_hdb].index)
    print("Filter out HDB with size > 2500 sqft (outlier detection)")
    display(df_dropped)
    
    # Target label is used below for outlier detection. Appropriately tagged data is dropped entirely.
    if 'price' in df:
    # Filtering those data with less than $200/square feet
        df['price per sq ft'] = df['price']/df['size_sqft']
        filter_price_sqft_200 = ((df['price per sq ft'] < 200) & (df['price per sq ft'] > 0))
        df_dropped = df[filter_price_sqft_200]
        df = df.drop(df[filter_price_sqft_200].index)
        print("Filtering those data with less than $200/square feet")
        display(df_dropped)
    # Filtering those data with less than 500 square feet, and more than $5000 per square feet
        df['price per sq ft'] = df['price']/df['size_sqft']
        filter_size = ((df['price per sq ft'] > 5000) & (df['price per sq ft'] > 0) & (df['size_sqft'] < 500))
        df_dropped = df[filter_size]
        df = df.drop(df[filter_size].index)
        print("Filtering those data with less than 500 square feet, and more than $5000 per square feet")
        display(df_dropped)
    # Filter out HDB more than $1600 per square feet and since last sold highest price for hdb is $1400
        df['price per sq ft'] = df['price']/df['size_sqft']
        filter_price_hdb_2 = ( (df['price per sq ft']>1600) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
        
        df_dropped = df[filter_price_hdb_2]
        df = df.drop(df[filter_price_hdb_2].index)
        display(df_dropped)
           
        df.drop('price per sq ft', axis=1, inplace=True)
    
    return df

In [8]:
def handle_missing_values(df, mode='train') -> pd.DataFrame:
    df = df.copy()
    # Treat missing year data as new.
    # Semantically, we define this attribute as the depreciation factor for pricing.
    # A new house or one with missing data denotes the depreciation factor is 0 or unknown.
    # The depreciation factor is assumed to be the difference between construction and current year.
    # TODO: Maybe do not treat future years as current! Inflation factor might be one to look out for.
    df['built_year'] = df['built_year'].fillna(2022)
    
    # TODO: 80 are missing. Should we remove them or should we keep it as 0?
    # Verify assumption if studio qualifies as 1 bed. 
    # 75 of missing are studio, we replace the Nan as 1
    filter_beds_studio = ((df.num_beds.isna()) & ((df.title.str.contains('studio','Studio', flags=re.IGNORECASE, regex=True))))
    df.loc[filter_beds_studio, "num_beds"] = 1
 
    if mode=='train':
        # 5 of missing, we do not have much info. Dropping these 5 data
        filter_drop_beds = df.num_beds.isna()
        df_dropped = df[filter_drop_beds]
        df = df.drop(df[filter_drop_beds].index)
        print("Dropped missing beds")
        display(df_dropped)

        # TODO: 400 are missing. Cannot remove so many data. Use 0 to denote absence of attribute.
        filter_drop_baths = df.num_baths.isna()
        df = df.drop(df[filter_drop_baths].index)
    elif mode=='test':
        # TODO: Filling with 0 for test data. Some imputation might be better.
        df['num_beds'] = df['num_beds'].fillna(0)
        df['num_baths'] = df['num_baths'].fillna(0)
     
    # Handle missing coordinates data
    for idx in df.index:
        if pd.isnull(df.at[idx, 'lat']) or pd.isnull(df.at[idx, 'lng']) or pd.isnull(df.at[idx, 'subzone']):
            lat, lng = find_lat_lng(df.at[idx, 'address'])
            if lat is not None:
                df.at[idx, 'lat'] = float(lat)
                #print("Fixed address {} with latitude: {}".format(df.at[idx, 'address'], df.at[idx, 'lat']))
            if lng is not None:
                df.at[idx, 'lng'] = float(lng)
                #print("Fixed address {} with longitude: {}".format(df.at[idx, 'address'], df.at[idx, 'lng']))
    
    return df

In [9]:
def transform_data(df) -> pd.DataFrame:
    df = df.copy()
    # TODO: Test against not doing this.
    df.loc[df["built_year"] > 2022, "built_year"] = 2022
    
    # Convert built_year into the aforementioned depreciation factor
    df["depreciation"] = (2022-df["built_year"])
    '''
    # TODO: Add details on why we are doing so
    df.loc[df.property_type.str.contains('hdb', flags=re.IGNORECASE, regex=True), 'property_type'] = 'hdb'
    df.loc[df.property_type.str.contains('condo', flags=re.IGNORECASE, regex=True), 'property_type'] = 'condo'
    df['property_type'] = df['property_type'].str.lower()
    df.loc[df.property_type.str.contains('cluster house', flags=re.IGNORECASE, regex=True), 'property_type'] = 'landed'
    df.loc[df.property_type.str.contains('townhouse', flags=re.IGNORECASE, regex=True), 'property_type'] = 'landed'
    df.loc[df.property_type.str.contains('land only', flags=re.IGNORECASE, regex=True), 'property_type'] = 'landed'
    df.loc[df.property_type.str.contains('apartment',  flags=re.IGNORECASE, regex=True), 'property_type'] = 'condo'
    df.loc[df.property_type.str.contains('bungalow', flags=re.IGNORECASE, regex=True), 'property_type'] = 'bungalow'
    df.loc[df.property_type.str.contains('semi-detached house', flags=re.IGNORECASE, regex=True), 'property_type'] = 'corner'
    df.loc[df.property_type.str.contains('corner terrace',flags=re.IGNORECASE, regex=True), 'property_type'] = 'corner'
    df.loc[df.property_type.str.contains('shophouse', flags=re.IGNORECASE, regex=True), 'property_type'] = 'protected'
    df.loc[df.property_type.str.contains('conservation house', flags=re.IGNORECASE, regex=True), 'property_type'] = 'protected'
    
    # Get one hot encoding of columns property_type
    one_hot = pd.get_dummies(df['property_type'])
    # Join the encoded df
    df = df.join(one_hot)
    '''
    '''
    df['tenure'] = df['tenure'].fillna(value=df.property_type)
    df.loc[df.tenure.str.contains('hdb', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('condo', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('terraced house', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('corner', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('landed', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('protected', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('bungalow', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'

    df.loc[df.tenure.str.contains('110-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('103-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('102-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('100-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'

    df.loc[df.tenure.str.contains('999-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('946-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('956-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('947-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('929-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'

    df['encoded_tenure'] = 0
    df.loc[df.tenure.str.contains('freehold', flags=re.IGNORECASE, regex=True), 'encoded_tenure'] = 1
    '''
    return df

In [10]:
def augment_data(df, distance=1) -> pd.DataFrame:
    df = df.copy()
    # get the files containing the infrastructures data
    cc = pd.read_csv("data/auxiliary-data/sg-commerical-centres.csv")
    mrt = pd.read_csv("data/auxiliary-data/sg-mrt-stations.csv")
    ps = pd.read_csv("data/auxiliary-data/sg-primary-schools.csv")
    ss = pd.read_csv("data/auxiliary-data/sg-secondary-schools.csv")
    sm = pd.read_csv("data/auxiliary-data/sg-shopping-malls.csv")
    sz = pd.read_csv("data/auxiliary-data/sg-subzones.csv")

    # calculate distance to nearest commercial center
    df['dist_2_nearest_cc'] = df.apply(
        lambda row: find_nearest_distance(row['lat'], row['lng'], cc), 
        axis=1)
    df['dist_2_nearest_cc'] = df['dist_2_nearest_cc'].round(decimals=3)

    # calculate the number of commercial centers within x km
    df['nearest_cc_count'] = df.apply(
        lambda row: count_nearest(row['lat'], row['lng'], cc, distance), 
        axis=1)
    
    # calculate distance to nearest mrt station
    df['dist_2_nearest_mrt'] = df.apply(
        lambda row: find_nearest_distance(row['lat'], row['lng'], mrt), 
        axis=1)
    df['dist_2_nearest_mrt'] = df['dist_2_nearest_mrt'].round(decimals=3)

    # calculate the number of mrt stations within x km
    df['nearest_mrt_count'] = df.apply(
        lambda row: count_nearest(row['lat'], row['lng'], mrt, distance), 
        axis=1)
    
    # calculate distance to nearest primary school
    df['dist_2_nearest_ps'] = df.apply(
        lambda row: find_nearest_distance(row['lat'], row['lng'], ps), 
        axis=1)
    df['dist_2_nearest_ps'] = df['dist_2_nearest_ps'].round(decimals=3)

    # calculate the number of primary schools within x km
    df['nearest_ps_count'] = df.apply(
        lambda row: count_nearest(row['lat'], row['lng'], ps, distance), 
        axis=1)
    # calculate distance to nearest secondary school
    df['dist_2_nearest_ss'] = df.apply(
        lambda row: find_nearest_distance(row['lat'], row['lng'], ss), 
        axis=1)
    df['dist_2_nearest_ss'] = df['dist_2_nearest_ss'].round(decimals=3)

    # calculate the number of secondary schools within x km
    df['nearest_ss_count'] = df.apply(
        lambda row: count_nearest(row['lat'], row['lng'], ss, distance), 
        axis=1)
    
    # calculate distance to nearest shopping mall
    df['dist_2_nearest_sm'] = df.apply(
        lambda row: find_nearest_distance(row['lat'], row['lng'], sm), 
        axis=1)
    df['dist_2_nearest_sm'] = df['dist_2_nearest_sm'].round(decimals=3)

    # calculate the number of secondary schools within x km
    df['nearest_sm_count'] = df.apply(
        lambda row: count_nearest(row['lat'], row['lng'], sm, distance), 
        axis=1)

    return df

In [11]:
def ignore_attributes(df) -> pd.DataFrame:
    df = df.copy()
    # Drop listing id; nominal identifier with no meaning
    df.drop('listing_id', axis=1, inplace=True)

    # Drop elevation; all the values are 0, spurious attribute
    df.drop('elevation', axis=1, inplace=True)

    # Drop url; nominal identifier with no meaning; useful for manual lookups or scraping
    df.drop('property_details_url', axis=1, inplace=True)

    # Drop floor level as 83% missing and sparse with the rest of the values. 
    # Not enough data available to get the model trained.
    df.drop('floor_level', axis=1, inplace=True)

    # Drop column property_type and tenure as it is now encoded
    df.drop('property_type',axis = 1, inplace=True)
    df.drop('tenure',axis=1, inplace=True)

    # Drop latitude and longitude; using it in conjunction with auxiliary data
    # Actual values are not semantically useful
    df.drop('lat', axis=1, inplace=True)
    df.drop('lng', axis=1, inplace=True)
    # Drop planning area to avoid dimensional bloat
    # Categories replaced with numerical identifiers from auxiliary data
    df.drop('planning_area', axis=1, inplace=True)
    # Address used in conjunction with augmentation
    df.drop('address', axis=1, inplace=True)

    # Drop this attribute as a significant amount of data(27.9%) is missing here.
    # We are unable to make much sense of this attribute.
    df.drop('total_num_units', axis=1, inplace=True) 

    # Transformed year into a depreciation factor already. Drop original attribute.
    df.drop('built_year', axis=1, inplace=True)

    # Title data is non-standard and unstructured with information already available in individual attributes.
    df.drop('title', axis=1, inplace=True)
    
    # Property name is assumed to have no inherent effect on the value of a listing.
    df.drop('property_name', axis=1, inplace=True)

    # BELOW CODE IN THIS SECTION IS ONLY MEANT TO GET THE SKELETON WORKING; RE-EVALUATE EACH ATTRIBUTE ONE BY ONE
    df.drop('furnishing', axis=1, inplace=True)
    df.drop('available_unit_types', axis=1, inplace=True)

    return df

In [12]:
def pre_process(df, mode='train') -> pd.DataFrame:
    if mode=='train': 
        df = remove_invalid_values(df)
        df = remove_outliers(df)
    df = handle_missing_values(df, mode)
    df = transform_data(df)
    df = augment_data(df, distance=RADIUS_OF_INFLUENCE_KM)
    df = ignore_attributes(df)
    
    return df

In [ ]:
# You may skip this block if you are not interested in Section 3.

df_preprocessed = pre_process(df, mode='train')
print(df_preprocessed.shape)
df_preprocessed.head()

Filterout HDB prices more than 2,000,000


,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
3066,296298,hdb flat for sale in 260 bangkit road,bukit batok / bukit panjang / choa chu kang (d23),bukit panjang neighbourhood centre,Hdb 4 Rooms,99-year leasehold,1989.0,4.0,4.0,1367,...,unspecified,"3, 4 br",NaN,https://www.99.co/singapore/hdb/bukit-panjang-...,1.377563,103.773332,0,bangkit,bukit panjang,2.100000e+06
3430,265512,hdb flat for sale in 789 choa chu kang north 6,bukit batok / bukit panjang / choa chu kang (d23),789 choa chu kang north 6,hdb 5 rooms,NaN,1996.0,4.0,2.0,1453,...,unspecified,NaN,87.0,https://www.99.co/singapore/hdb/789-choa-chu-k...,1.396419,103.751106,0,choa chu kang north,choa chu kang,8.400000e+06
5976,385586,hdb flat for sale in 238 compassvale walk,hougang / punggol / sengkang (d19),compassvale plains,hdb 4 rooms,99-year leasehold,1999.0,1.0,10.0,232,...,unspecified,"1, 3, 4 br",NaN,https://www.99.co/singapore/hdb/compassvale-pl...,1.390556,103.898921,0,sengkang town centre,sengkang,3.924243e+10
16264,287109,hdb flat for sale in 238 compassvale walk,hougang / punggol / sengkang (d19),compassvale plains,hdb 4 rooms,99-year leasehold,1999.0,1.0,10.0,129,...,unspecified,"1, 3, 4 br",NaN,https://www.99.co/singapore/hdb/compassvale-pl...,1.390556,103.898921,0,sengkang town centre,sengkang,4.985919e+09


Filter out HDB with more bathrooms than bedrooms


,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
1213,181286,hdb flat for sale in 310 canberra road,sembawang / yishun (d27),310 canberra road,hdb,NaN,1998.0,3.0,4.0,1542,...,unspecified,NaN,86.0,https://www.99.co/singapore/hdb/310-canberra-r...,1.447000,103.822080,0,sembawang central,sembawang,1248300.0
2221,357282,hdb flat for sale in 135 pasir ris street 11,pasir ris / tampines (d18),hdb-pasir ris,hdb executive,99-year leasehold,1994.0,3.0,4.0,1603,...,unfurnished,"studio, 1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbpasir-ris-d...,1.364586,103.957437,0,pasir ris drive,pasir ris,787500.0
2233,514384,hdb flat for sale in 302b anchorvale link,hougang / punggol / sengkang (d19),anchorvale court,hdb 2 rooms,99-year leasehold,2007.0,1.0,2.0,473,...,partial,"1, 2, 3, 4 br",NaN,https://www.99.co/singapore/hdb/anchorvale-cou...,1.387977,103.890863,0,anchorvale,sengkang,315000.0
2578,252034,hdb flat for sale in 109 tampines street 11,pasir ris / tampines (d18),109 tampines street 11,hdb 3 rooms,NaN,1984.0,1.0,2.0,796,...,unspecified,NaN,181.0,https://www.99.co/singapore/hdb/109-tampines-s...,1.347408,103.948812,0,tampines east,tampines,481100.0
4152,726218,hdb flat for sale in 236 lorong 1 toa payoh,balestier / toa payoh (d12),hdb-toa payoh,hdb executive,99-year leasehold,1986.0,1.0,2.0,1539,...,unspecified,"studio, 1, 2, 3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/hdbtoa-payoh-d...,1.340336,103.852014,0,braddell,toa payoh,1113000.0
4274,476726,hdb flat for sale in 486 admiralty link,sembawang / yishun (d27),sembawang green,hdb,99-year leasehold,2004.0,2.0,3.0,969,...,unspecified,"2, 3 br",NaN,https://www.99.co/singapore/hdb/sembawang-gree...,1.454984,103.817403,0,sembawang north,sembawang,493500.0
4417,394582,hdb flat for sale in 702 west coast road,buona vista / west coast / clementi (d5),702 west coast road,hdb,NaN,1980.0,1.0,2.0,731,...,unspecified,NaN,192.0,https://www.99.co/singapore/hdb/702-west-coast...,1.307220,103.761206,0,clementi west,clementi,357000.0
6878,482001,hdb flat for sale in 436 woodlands street 41,admiralty / woodlands (d25),hdb-woodlands,Hdb 4 Rooms,99-year leasehold,1996.0,1.0,2.0,904,...,unspecified,"1, 2, 3, 4, 5, 6, 7 br",NaN,https://www.99.co/singapore/hdb/hdbwoodlands-d...,1.429041,103.772075,0,woodgrove,woodlands,498800.0
7471,316210,hdb flat for sale in 107a canberra street,sembawang / yishun (d27),eastbrook @ canberra,hdb,99-year leasehold,2017.0,2.0,3.0,1001,...,unspecified,"1, 2, 3 br",1030.0,https://www.99.co/singapore/hdb/eastbrook-canb...,1.449854,103.832972,0,sembawang east,sembawang,577500.0
8946,805066,hdb flat for sale in 714 clementi west street 2,buona vista / west coast / clementi (d5),714 clementi west street 2,hdb,NaN,1980.0,1.0,2.0,721,...,unspecified,NaN,196.0,https://www.99.co/singapore/hdb/714-clementi-w...,1.303990,103.761893,0,clementi west,clementi,407400.0


Filter out HDB with more than 4 bathrooms or 6 bedrooms


...

Filtering those data with less than 500 square feet, and more than $5000 per square feet


,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price,price per sq ft
635,432676,3 bed house for sale in the glencaird residences,dalvey road,the glencaird residences,Land Only,freehold,2000.0,3.0,3.0,107,...,"3, 4, 5, 6 br",12.0,https://www.99.co/singapore/houses/the-glencai...,1.315477,103.819725,0,nassim,tanglin,101640000.0,949906.542056
930,235254,hdb flat for sale in 929 tampines street 91,pasir ris / tampines (d18),tampines palmspring,hdb 4 rooms,99-year leasehold,1984.0,3.0,2.0,104,...,"1, 2, 3, 4 br",NaN,https://www.99.co/singapore/hdb/tampines-palms...,1.346378,103.939618,0,tampines west,tampines,575400.0,5532.692308
949,190331,hdb flat for sale in 16 toh yi drive,clementi park / upper bukit timah (d21),toh yi gardens,Hdb 4 Rooms,99-year leasehold,1988.0,3.0,2.0,104,...,"2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/toh-yi-gardens...,1.339369,103.772544,0,anak bukit,bukit timah,732900.0,7047.115385
2177,358443,hdb flat for sale in 121c canberra street,sembawang / yishun (d27),eastbank @ canberra,hdb 5 rooms,99-year leasehold,2016.0,3.0,2.0,113,...,3 br,372.0,https://www.99.co/singapore/hdb/eastbank-canbe...,1.447378,103.834366,0,sembawang east,sembawang,753900.0,6671.681416
4241,140317,hdb flat for sale in 291b bukit batok street 24,bukit batok / bukit panjang / choa chu kang (d23),spring view,hdb 4 rooms,99-year leasehold,1998.0,3.0,2.0,103,...,"2, 3, 4 br",NaN,https://www.99.co/singapore/hdb/spring-view-de...,1.343066,103.755985,0,bukit batok south,bukit batok,588000.0,5708.737864
4346,186305,hdb flat for sale in 144 potong pasir avenue 2,macpherson / potong pasir (d13),hdb-potong pasir,hdb 4 rooms,99-year leasehold,1984.0,4.0,2.0,103,...,"1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbpotong-pasi...,1.332650,103.865496,0,potong pasir,toa payoh,819000.0,7951.456311
7562,645542,hdb flat for sale in 2c boon tiong road,alexandra / commonwealth (d3),boon tiong arcadia,hdb 5 rooms,99-year leasehold,2001.0,3.0,2.0,107,...,"3, 4 br",NaN,https://www.99.co/singapore/hdb/boon-tiong-arc...,1.285968,103.830103,0,tiong bahru,bukit merah,1037400.0,9695.327103
9855,974261,hdb flat for sale in 319 bukit batok street 33,bukit batok / bukit panjang / choa chu kang (d23),hdb-bukit batok,Hdb 5 Rooms,99-year leasehold,1986.0,3.0,2.0,122,...,"studio, 1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbbukit-batok...,1.361270,103.746427,0,hong kah north,bukit batok,648900.0,5318.852459
9876,695422,hdb flat for sale in 614 yishun street 61,sembawang / yishun (d27),614 yishun street 61,hdb,NaN,1987.0,4.0,2.0,142,...,NaN,72.0,https://www.99.co/singapore/hdb/614-yishun-str...,1.419741,103.836178,0,yishun south,yishun,1050900.0,7400.704225
10804,949328,4 bed condo for sale in the lilium,33 how sun road,the lilium,condo,freehold,2021.0,4.0,3.0,129,...,"studio, 2, 3, 4, 5 br",80.0,https://www.99.co/singapore/condos-apartments/...,1.345552,103.881514,0,upper paya lebar,serangoon,2602700.0,20175.968992


,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price,price per sq ft
2320,929124,hdb flat for sale in 406 jurong west street 42,boon lay / jurong / tuas (d22),hong kah court,hdb 3 rooms,99-year leasehold,1984.0,3.0,2.0,104,...,"3, 4, 5 br",NaN,https://www.99.co/singapore/hdb/hong-kah-court...,1.354302,103.722454,0,hong kah,jurong west,493500.0,4745.192308
7000,525841,hdb flat for sale in 123 marsiling rise,admiralty / woodlands (d25),hdb-woodlands,hdb 5 rooms,99-year leasehold,1985.0,3.0,2.0,126,...,"1, 2, 3, 4, 5, 6, 7 br",NaN,https://www.99.co/singapore/hdb/hdbwoodlands-d...,1.439410,103.781158,0,woodlands regional centre,woodlands,564900.0,4483.333333
9744,660430,hdb flat for sale in 93 paya lebar way,macpherson / potong pasir (d13),93 paya lebar way,hdb 3 rooms,NaN,1972.0,2.0,1.0,624,...,NaN,145.0,https://www.99.co/singapore/hdb/93-paya-lebar-...,1.322728,103.883483,0,macpherson,geylang,1155000.0,1850.961538
12426,868860,hdb flat for sale in 273a jurong west avenue 3,boon lay / jurong / tuas (d22),wenya,hdb 5 rooms,99-year leasehold,2001.0,3.0,2.0,115,...,"studio, 1, 3, 4 br",NaN,https://www.99.co/singapore/hdb/wenya-de72jyws...,1.351438,103.703519,0,wenya,jurong west,567000.0,4930.434783
15465,818611,3 bed condo for sale in hdb-serangoon estate,262 serangoon central drive,hdb-serangoon estate,condo,99-year leasehold,1989.0,3.0,1.0,1119,...,"1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbserangoon-e...,1.353518,103.872448,0,serangoon central,serangoon,1907800.0,1704.915103


Dropped missing beds


,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
3915,243690,hdb flat for sale in 417 eunos road 5,eunos / geylang / paya lebar (d14),417 eunos road 5,hdb,NaN,1984.0,NaN,2.0,1151,...,unspecified,NaN,116.0,https://www.99.co/singapore/hdb/417-eunos-road...,1.318176,103.896902,0,geylang east,geylang,819000.0
4939,831032,hdb flat for sale in 539 bedok north street 3,bedok / upper east coast (d16),539 bedok north street 3,hdb,NaN,1980.0,NaN,NaN,731,...,unspecified,NaN,112.0,https://www.99.co/singapore/hdb/539-bedok-nort...,1.331612,103.924833,0,bedok north,bedok,367500.0
5002,730482,hdb flat for sale in 205a punggol field,hougang / punggol / sengkang (d19),205a punggol field,Hdb,NaN,2004.0,NaN,NaN,1184,...,unspecified,NaN,90.0,https://www.99.co/singapore/hdb/205a-punggol-f...,1.403418,103.896423,0,matilda,punggol,669900.0
11500,120518,hdb flat for sale in 247 bukit batok east aven...,bukit batok / bukit panjang / choa chu kang (d23),gombak view,hdb,99-year leasehold,1985.0,NaN,NaN,786,...,unspecified,"studio, 2, 3, 4 br",NaN,https://www.99.co/singapore/hdb/gombak-view-de...,1.349904,103.756987,0,bukit batok east,bukit batok,409500.0
19866,254804,hdb flat for sale in 224b sumang lane,hougang / punggol / sengkang (d19),matilda edge,Hdb,99-year leasehold,2017.0,NaN,2.0,1001,...,unspecified,"studio, 1, 2, 3 br",1330.0,https://www.99.co/singapore/hdb/matilda-edge-d...,1.401793,103.893175,0,matilda,punggol,609000.0


## 2c. Post-process

Here, we separately process the training and validation sets. Since we perform aggregation-based methods here, this is done after split to avoid information leakage and ensures salient evaluation results.

If you wish to explore each model and go through our data handling, proceed to Section 3 - Model Evaluation. Alternately, if you are only interested in the final output, you may jump to Section 4 - Prediction to gather results.

In [ ]:
def target_encoding_location(df, mode='train', aggregate=None):
    # Target encode subzone based on price.
    cols = ('subzone')
    df.loc[:,cols] = df.loc[:,cols].fillna('null')
    if mode == 'train':
        aggregate = df['price'].groupby(df.loc[:,cols]).median()
    for idx in df.index:
        if df.at[idx, cols] in aggregate:
            df.at[idx, cols] = float(aggregate[df.at[idx, cols]])
        else:
            df.at[idx, cols] = float(aggregate['null'])
    return df, aggregate

def standardize(df, mode='train', scaler=None):
    # Normalize values here.
    cols = ('size_sqft', 'dist_2_nearest_cc', 'dist_2_nearest_mrt', 'dist_2_nearest_ps', 'dist_2_nearest_ss', 'dist_2_nearest_sm', 'subzone')
    if scaler is None:
        scaler = StandardScaler()
    if mode=='train':
        df.loc[:,cols] = scaler.fit_transform(df.loc[:,cols])
    elif mode=='test':
        df.loc[:,cols] = scaler.transform(df.loc[:,cols])
    return df, scaler
    

def post_process(df, mode='train', aggregate=None, scaler=None):
    df, aggregate = target_encoding_location(df, mode, aggregate)
    df, scaler = standardize(df, mode, scaler)
    return df, aggregate, scaler


In [ ]:
# You may skip this block if you are not interested in Section 3.

# First we create our train and validation split to ensure there is no information leakage from validation set to train set
train_set, val_set = train_test_split(df_preprocessed, train_size=TRAIN_SIZE, random_state=RANDOM_CONTROL, shuffle=True)

# Then, we run post-processing on train set and use it to post-process validation set.
train_set, aggregate, scaler = post_process(train_set, mode='train')
val_set, _, _ = post_process(val_set, mode='test', aggregate=aggregate, scaler=scaler)

# Check for why we are removing Mandai Estate!!

# 3. Model Evaluation

Here, we evaluate models for their predictions. RMSE is the metric used for comparison. The following models are evaluated:

* Linear Regression
* Random Forest
* Gradient Boosting
* AdaBoost
* Extra Trees Regressor
* Bagging Regressor
* Neural Net

This section also involves hyperparameter tuning wherein we search for best settings per model. Those settings are later used to set hyperparameters as in Section 1 and dictate the final predictions in Section 4. 

In [ ]:
def validate(y, y_hat) -> None:
    rmse = mean_squared_error(y, y_hat, squared=False)
    print('Validation RMSE: {:.3}'.format(rmse))
    return

# Need to recreate whole set for Section 3. This is a 'hack' to get GridSearch to work with cross-validation.
# To ensure that we get consistent results across multiple iterations, we need to fix the train-test split.
# Later we run GridSearch on entire set by passing in our predefined split as parameter. 
df_postprocessed = pd.concat([train_set, val_set], ignore_index=True)
X_housing = df_postprocessed.loc[:, df_postprocessed.columns != 'price']
y_housing = df_postprocessed['price']
X_train = train_set.loc[:, train_set.columns != 'price']
y_train = train_set['price']
X_val = val_set.loc[:, val_set.columns != 'price']
y_val = val_set['price']

# Use our predefined split for GridSearch below.
val_split_indices = [-1 if x in X_train.index else 0 for x in X_housing.index]
ps = PredefinedSplit(test_fold=val_split_indices)

## 3a. Linear Regression

In [ ]:
model_lr = train(X_train, y_train, model_name='lr')
y_hat_lr = predict(model_lr, X_val)
validate(y_val, y_hat_lr)

## 3b. Random Forest

In [ ]:
def bestfit_rf(X_feature, y_label, train_test_split):
    estimator = RandomForestRegressor()
    params = {'n_estimators': [25, 50, 100, 200],
              'max_depth': [5, 10, 20, 30, 40, 50],
              'min_samples_split': [2, 3, 4, 5],
              'min_samples_leaf': [1, 2, 3, 4],
              'criterion': ["squared_error"],
              'max_features': [1, 2, 3],
              'bootstrap': [True, False],
              'random_state': [RANDOM_CONTROL]}
    model_rf = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_rf.fit(X_feature, y_label)
    print('Random Forest Best Parameters: {}'.format(model_rf.best_params_))
    return model_rf

model_rf = bestfit_rf(X_housing, y_housing, ps)
y_hat_rf = predict(model_rf, X_val)
validate(y_val, y_hat_rf)

## 3c. Gradient Boosting

In [ ]:
def bestfit_gb(X_feature, y_label, train_test_split):
    estimator = GradientBoostingRegressor()
    params = {'n_estimators': [25, 50, 100, 200],
              'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
              'max_depth': [5, 10, 20, 30, 40, 50],
              'min_samples_split': [2, 3, 4, 5],
              'min_samples_leaf': [1, 2, 3, 4],
              'criterion': ["squared_error", "friedman_mse"],
              'max_features': [1, 2, 3],
              'random_state': [RANDOM_CONTROL]}
    model_gb = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_gb.fit(X_feature, y_label)
    print('Gradient Boost Best Parameters: {}'.format(model_gb.best_params_))
    return model_gb
    

model_gb = bestfit_gb(X_housing, y_housing, ps)
y_hat_gb = predict(model_gb, X_val)
validate(y_val, y_hat_gb)

## 3d. AdaBoost

In [ ]:
def bestfit_ab(X_feature, y_label, train_test_split):
    base_estimator = DecisionTreeRegressor()
    estimator = AdaBoostRegressor(base_estimator=base_estimator)
    params = {'base_estimator__max_depth': [5, 10, 20, 30, 40, 50],
              'base_estimator__splitter': ['best', 'random'],
              'n_estimators': [25, 50, 100, 200],
              'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.5, 1],
              'random_state': [RANDOM_CONTROL]}
    model_ab = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_ab.fit(X_housing, y_housing)
    print('AdaBoost Best Parameters: {}'.format(model_ab.best_params_))
    return model_ab


model_ab = bestfit_ab(X_housing, y_housing, ps)
y_hat_ab = predict(model_ab, X_val)
validate(y_val, y_hat_ab)

## 3e. Extra Trees Regressor

In [ ]:
def bestfit_et(X_feature, y_label, train_test_split):
    estimator = ExtraTreesRegressor()
    params = {'n_estimators': [25, 50, 100],
              'max_depth': [5, 10, 25, 50],
              'min_samples_split': [2],
              'min_samples_leaf': [1],
              'criterion': ["squared_error", "friedman_mse"],
              'max_features': [1],
              'bootstrap': [True, False],
              'random_state': [RANDOM_CONTROL]}
    model_et = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_et.fit(X_housing, y_housing)
    print('Extra Trees Best Parameters: {}'.format(model_et.best_params_))
    return model_et


model_et = bestfit_et(X_housing, y_housing, ps)
y_hat_et = predict(model_et, X_val)
validate(y_val, y_hat_et)

## 3f. Bagging Regressor

In [ ]:
def bestfit_br(X_feature, y_label, train_test_split):
    base_estimator = DecisionTreeRegressor()
    estimator = BaggingRegressor(base_estimator=base_estimator)
    params = {'n_estimators': [25, 50, 100],
              'max_features': [1],
              'random_state': [RANDOM_CONTROL]}
    model_br = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=ps)
    model_br.fit(X_housing, y_housing)
    print('Bagging Best Parameters: {}'.format(model_br.best_params_))
    return model_br


model_br = bestfit_br(X_housing, y_housing, ps)
y_hat_br = predict(model_br, X_val)
validate(y_val, y_hat_br)

## 3g. Neural Net

In [ ]:
X_housing = X_housing.to_numpy()
y_housing = y_housing.to_numpy()
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()

#print(X_train.shape)
#print(y_train.shape)
train_dataloader = DataLoader([ [X_train[i], y_train[i]] for i in range(len(X_train)) ], batch_size=NN_BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
# Define MLP architecture
class Model(nn.Module):
    
    def __init__(self, device='cpu'):
        super(Model, self).__init__()
        self.device = device
        
        # Modify accordingly
        self.fc1 = nn.Linear(1, 4)
        self.fc2 = nn.Linear(4, 16)
        self.fc3 = nn.Linear(16, 64)
        self.fc4 = nn.Linear(64, 4)
        self.fc5 = nn.Linear(4, 1)
        self.relu = nn.ReLU()
        
        self.dense = nn.Sequential(self.fc1, self.relu, self.fc2, self.relu, self.fc3, self.relu, 
                                   self.fc4, self.relu, self.fc5)
        
    def forward(self, x):
        pred = self.dense(x)
        return pred
    
# Train
device = 'cpu'
model = Model()
optimizer = optim.Adam(model.parameters(), NN_LEARNING_RATE)
criterion = nn.MSELoss()
model.to(device)
for epoch in range(NN_NUM_EPOCHS):
    running_loss = 0
    for idx, (x_features, y_labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        x_features = x_features.to(device, dtype=torch.float)
        y_labels = y_labels.to(device, dtype=torch.float)
        prediction = model(x_features)
        loss = torch.sqrt(criterion(prediction, y_labels)) # Standardize RMSE loss
        loss.backward()
        optimizer.step()
        running_loss += loss
        if (idx+1) %100 == 0: 
            running_loss = format(running_loss/100, '.4f')
            print(f"Epoch [{epoch+1} Batches processed | {idx}] Loss: {running_loss}")
            running_loss = 0
print("Finished Training.")


In [ ]:
# Validate
X_features = torch.from_numpy(X_val)
y_labels = torch.from_numpy(y_val)
X_features = X_features.to(device, dtype=torch.float)
y_labels = y_labels.to(device, dtype=torch.float)
prediction = model(X_features)
rmse_nn = torch.sqrt(criterion(prediction, y_labels))

print('Neural Net Validation rmse: {:.3}'.format(rmse_nn))

# 4. Prediction

Now that we are done evaluating each model and comparing their performance, we select the model that performed best on the validation set to generate our overall predictions. We re-train this model from scratch on the whole set of training data here as it is assumed ready to ship so we wish to use as much data for training as we can. We later test this model against the provided test data and generate a CSV file of predicted values.

In [ ]:
# Train over entire training set
df_train = pd.read_csv('data/train.csv') 
df_train = pre_process(df_train, mode='train')
df_train, aggregate, scaler = post_process(df_train, mode='train')
X_train = df_train.loc[:, df_train.columns != 'price']
y_train = df_train['price']
model = train(X_train, y_train, model_name=BEST_MODEL) # Choose your model from {'lr', 'rf', 'gb', ab', 'et', 'br', 'nn'}

# Predict labels for test set
df_test = pd.read_csv('data/test.csv') 
df_test = pre_process(df_test, mode='test')
df_test, _, _ = post_process(df_test, mode='test', aggregate=aggregate, scaler=scaler)
predictions = predict(model, df_test)
predictions = pd.DataFrame(predictions)
predictions.to_csv('predictions.csv', header=['Predicted'], index=True, index_label='Id')